# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = '../WeatherPy/output_data/cities.csv'
weather_df = pd.read_csv(weather_file)
weather_df.dtypes

City                 object
Temperature (F)     float64
Humidity (%)          int64
Clouds(% Cloudy)      int64
Windspeed (mph)     float64
Latitude            float64
Longitude           float64
Country              object
Date-Time             int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
locations = weather_df[["Latitude","Longitude"]].astype(float)
humidity = weather_df["Humidity (%)"].astype(float)

In [5]:
fig = gmaps.figure(center=(20,0),zoom_level=1.75)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perfect_destination = weather_df.loc[(weather_df["Temperature (F)"] < 80) & (weather_df["Temperature (F)"] > 65) 
                                     & (weather_df["Humidity (%)"] > 40) & (weather_df["Humidity (%)"] < 75)
                                     & (weather_df["Windspeed (mph)"] < 7) & (weather_df["Clouds(% Cloudy)"] < 25)]
perfect_destination

,City,Temperature (F),Humidity (%),Clouds(% Cloudy),Windspeed (mph),Latitude,Longitude,Country,Date-Time
26,manutuke,72.00,61,10,3.00,-38.6833,177.9167,NZ,1612734031
36,gwadar,65.43,56,0,6.64,25.1216,62.3254,PK,1612734043
42,hilo,75.49,50,1,3.44,19.7297,-155.0900,US,1612733521
78,kapaa,74.32,56,1,3.44,22.0752,-159.3190,US,1612733619
206,chai badan,75.70,65,0,1.05,15.2000,101.1333,TH,1612734243
287,la palma,66.85,51,1,3.44,33.8464,-118.0467,US,1612734331
302,tacuarembo,79.38,51,0,3.94,-31.7333,-55.9833,UY,1612734357
317,makakilo city,77.40,41,1,6.91,21.3469,-158.0858,US,1612734077
343,dourados,72.27,70,0,3.11,-22.2211,-54.8056,BR,1612734406
364,cananeia,74.37,67,0,5.99,-25.0147,-47.9267,BR,1612734431


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
perfect_destination["Hotel Name"]= ""
hotel_df = perfect_destination
hotel_df = hotel_df.drop(columns=["Temperature (F)","Humidity (%)","Clouds(% Cloudy)","Windspeed (mph)","Date-Time"])
hotel_df

C:\Users\SageTeam\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Latitude,Longitude,Country,Hotel Name
26,manutuke,-38.6833,177.9167,NZ,
36,gwadar,25.1216,62.3254,PK,
42,hilo,19.7297,-155.0900,US,
78,kapaa,22.0752,-159.3190,US,
206,chai badan,15.2000,101.1333,TH,
287,la palma,33.8464,-118.0467,US,
302,tacuarembo,-31.7333,-55.9833,UY,
317,makakilo city,21.3469,-158.0858,US,
343,dourados,-22.2211,-54.8056,BR,
364,cananeia,-25.0147,-47.9267,BR,


In [8]:
# find a hotel in each city

for index, row in hotel_df.iterrows():
    lat = hotel_df["Latitude"][index]
    lng = hotel_df["Longitude"][index]
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type": "lodging",
        "key": gkey}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(base_url, params=params).json()
    hotel_name = response["results"][0]["name"]
    hotel_df["Hotel Name"][index] = hotel_name

hotel_df

C:\Users\SageTeam\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,City,Latitude,Longitude,Country,Hotel Name
26,manutuke,-38.6833,177.9167,NZ,Jolly Stockman Hotel
36,gwadar,25.1216,62.3254,PK,Sadaf Resort
42,hilo,19.7297,-155.0900,US,Hilo Hawaiian Hotel
78,kapaa,22.0752,-159.3190,US,Sheraton Kauai Resort at Coconut Beach
206,chai badan,15.2000,101.1333,TH,Narai Grand Hotel
287,la palma,33.8464,-118.0467,US,La Quinta Inn & Suites by Wyndham Buena Park
302,tacuarembo,-31.7333,-55.9833,UY,Hotel Orange
317,makakilo city,21.3469,-158.0858,US,Marriott's Ko Olina Beach Club
343,dourados,-22.2211,-54.8056,BR,Bravo City Hotel
364,cananeia,-25.0147,-47.9267,BR,Casa azul


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Create bank symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig = gmaps.figure(center=(0,0),zoom_level=1.75)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))